<a href="https://colab.research.google.com/github/phreakyphoenix/PrimeFinding/blob/master/Prime_Finding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Comparing Prime Finding Algorithms
Inspiration [video](https://www.youtube.com/watch?v=fwxjMKBMR7s) and [reddit post](https://www.reddit.com/r/learnprogramming/comments/1apk2dk/help_needed_with_recreating_dijkstras_prime/)

In [1]:
import time, heapq
import pandas as pd

In [2]:
# simple trial division
def trial_division(n):
    primes = []
    for i in range(2, n):
        is_prime = True
        for j in range(2, int(i**0.5) + 1):
            if i % j == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(i)
    return primes

In [3]:
# sieve of eratosthenes method of finding primes
def sieve_of_eratosthenes(n):
    primes = []
    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False

    for i in range(2, int(n**0.5) + 1):
        if is_prime[i]:
            primes.append(i)
            for j in range(i*i, n + 1, i):
                is_prime[j] = False

    for i in range(int(n**0.5) + 1, n + 1):
        if is_prime[i]:
            primes.append(i)

    return primes

In [4]:
# original,
def dijkstraPrimes(n):
    pool = [[4,2]]
    primes = [2]
    for i in range(3, n):
        if min(pool)[0] > i:
            pool.append([i**2,i])
            primes+=[i]
        else:
            for pair in pool:
                while pair[0] <= i:
                    pair[0] += pair[1]
    return primes

In [5]:
# level 1, optimized version of dijkstraPrimes using heapq
def dijkstraPrimes_heapq(n):
    pool = [(4, 2)]
    primes = [2]

    for i in range(3, n):
        if pool[0][0] > i:
            heapq.heappush(pool, (i**2, i))
            primes.append(i)
        else:
            while pool[0][0] <= i:
                current_value, current_prime = heapq.heappop(pool)
                heapq.heappush(pool, (current_prime + current_value, current_prime))

    return primes

In [6]:
# level 2, optimized version of dijkstraPrimes_heapq, removing redundant access
def dijkstraPrimes_heapq_2(n):
    pool = [(4, 2)]
    primes = [2]

    for i in range(3, n):
        current_value, current_prime = pool[0]

        if current_value > i:
            heapq.heappush(pool, (i**2, i))
            primes.append(i)
        else:
            while current_value <= i:
                heapq.heappop(pool)
                heapq.heappush(pool, (current_prime + current_value, current_prime))
                current_value, current_prime = pool[0]

    return primes

In [7]:
#visual checks
print (trial_division(50))
print (sieve_of_eratosthenes(50))
print (dijkstraPrimes(50))
print (dijkstraPrimes_heapq(50))
print (dijkstraPrimes_heapq_2(50))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]


In [8]:
#length checks
assert  len(dijkstraPrimes(1000)) ==\
        len(dijkstraPrimes_heapq(1000)) ==\
        len(dijkstraPrimes_heapq_2(1000)) ==\
        len(sieve_of_eratosthenes(1000)) ==\
        len(trial_division(1000))

In [9]:
# benchmarking from https://www.reddit.com/r/learnprogramming/comments/1apk2dk/help_needed_with_recreating_dijkstras_prime/
def benchmark(func, *args, **kwargs):
    total_time = 0
    num_runs = 100
    start_time = time.time()
    for _ in range(num_runs):
        func(*args, **kwargs)
    end_time = time.time()
    total_time = end_time-start_time
    average_time = total_time / num_runs
    return (average_time,total_time)

In [10]:
# show the benchmarking result in a df
def benchmark_df(funcs, *args, **kwargs):
    results = {}
    for func in funcs:
        results[func.__name__] = benchmark(func, *args, **kwargs)
    return pd.DataFrame(results).T.rename(columns={0: 'Avg Time', 1: 'Total Time'})

In [11]:
#comparing all algorithms on 1000 iterations
benchmark_df([trial_division, sieve_of_eratosthenes, dijkstraPrimes, dijkstraPrimes_heapq, dijkstraPrimes_heapq_2], 1_000)

,Avg Time,Total Time
trial_division,0.000520,0.051987
sieve_of_eratosthenes,0.000083,0.008327
dijkstraPrimes,0.004390,0.439031
dijkstraPrimes_heapq,0.000759,0.075891
dijkstraPrimes_heapq_2,0.000810,0.081028


In [12]:
#comparing all efficient algorithms on 100k iterations
benchmark_df([trial_division, sieve_of_eratosthenes, dijkstraPrimes_heapq, dijkstraPrimes_heapq_2], 100_000)

,Avg Time,Total Time
trial_division,0.143241,14.324062
sieve_of_eratosthenes,0.016785,1.678544
dijkstraPrimes_heapq,0.153464,15.346443
dijkstraPrimes_heapq_2,0.136186,13.618629


In [14]:
#comparing 2 heap algorithms on 1M iterations
benchmark_df([trial_division, sieve_of_eratosthenes, dijkstraPrimes_heapq, dijkstraPrimes_heapq_2], 1_000_000)

,Avg Time,Total Time
trial_division,3.183875,318.387467
sieve_of_eratosthenes,0.104170,10.417039
dijkstraPrimes_heapq,1.742660,174.265993
dijkstraPrimes_heapq_2,1.745813,174.581320


### Conclusion
1. Sieve of Eratosthenes cannot be beaten on speed.
2. Trial division takes negligible aux space
3. Diskstra's algorithm is great middle ground, taking a bit more time compared to trial division, but much less space than the sieve.
4. All algorithms can be improved with human ingenuity.
5. **dijkstraPrimes_heapq_2** is the fastest amongst the Dijkstra-inspired prime finding implementations in this notebook.
6. On colab CPU, for 100k iterations, **dijkstraPrimes_heapq_2** was a tiny bit faster than trial_division, but not on mac m1. It's way faster on n>1M as expected.

In [14]:
# Compare mmemory space and effect of using gpus, maybe with TF later.